This notebook will showcase the capabilities of the Hugging Face **smolagents** library, which provides a lightweitght framework for creating AI agents.

The following code samples will exemplify how to use this package to build agents capable of searching for data, executing code, and interacting with web pages. Finally, we'll review how to combine multiple agents to create more powerful systems.

# Knowing *smolagents*

The package *smolagents* is a framework for building AI agents, providing LLMs with the agency to interact with the real world, like searching or generating images.

These are some of the main advantages of using this package for building your AI Agents:

* **Simplicity.** Has minimal code complexity and abstractions to make the framework easy to understand and use
* **Flexible LLM Support.** The package is capable of working with any LLM through integration with Hugging Face tools and external APIs
* **Code-First Approach.** It has first-class support for Code Agents that write their actions directly in code, removing the need for parsing and simplifying tool calling
* **HF Hub Integrations.** It also counts with seamless integration with the Hugging Face Hub, allowing the use of Gradio Sapces as tools

Another difference with other frameworks is that *smolagents* focuses on tool calls in code instead of writing actions in JSON format. This skips the need to parse the JSON data in order to build code that calls the tools, and executing it directly.

It is better to use this package under the following situations:

* You need a **lightweight and minimal solution**
* You want to **experiment quickly** without complex configurations
* Your **application logic is straightforward**

The package works with **multi-step agents**, where each perform the following:

* One thought
* One tool call and execution

The primary agent of the package is the **CodeAgent**, although it also supports **ToolCallingAgent**, which writes tool calls in JSON, like other frameworks.

It is important to mention that the package defines its tools with the *@tool* decorator, or using the *Tool* class.

The model integration of the package supports flexible connection with multiple LLM models that meet certain criteria. These are some of the predefined classes that allow model connection:

* **TransformersModel.** Implements local transformers pipelines for seamless integration
* **HfApiModel.** Supports serverless inference calls through the *Hugging Face's infrastructure*, or through *third-party inference providers*
* **LiteLLMModel.** Leverages *LiteLLM* for lightweight model interactions
* **OpenAIServerModel.** Connects to any service that offers an OpenAI API interface
* **AzureOpenAIServerModel.** Supports integration with any Azure OpenAI deployment

# Building Agents that Use Code

As mentioned before, the core agent type of *smolagents* is the **Code Agent** that generates Python tool calls to perform actions. This approach reduces the number of required actions, simplifies complex operations, and enables reuse of existing code functions.

The general approach that most of the frameworks follow is to use a JSON format to specify tool names and arguments as strings, which then the system **must parse to determine which tool to execute**. However, there are studies that suggest that **tool-calling LLMs work more effectively with code directly**, being some of the core advantages the following:

* **Composability.** Easily combine and reuse actions
* **Object Management.** Work directly with complex structures like images
* **Generality.** Express any computationally possible task
* **Natural for LLMs.** High-quality code is already present in LLM training data

A *CodeAgent* performs actions through a cycle of steps, with existing variables and knowledge being incorporated into the agent's context, which is kept in an execution log.

1. The system prompt is stored in a **SystemPromptStep**, and the user query is logged in a **TaskStep**
2. The following loop is executed:
    2.1 Method **agent.write_memory_to_message() writes the agent's logs into a list of LLM-readable chat messages
    2.2 These messages are sent to a **Model**, which generates a completion
    2.3 The completion is parsed to extract the action, which should be a code snippet
    2.4 The action is executed
    2.5 The results are logged into memory in an **ActionStep**

At the end of each step, if the agent includes any function calls in **agent.step_callback**, they are executed.

In [3]:
# Install the smolagents package
# pip install smolagents

import numpy as np
import time
import datetime

from huggingface_hub import login
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool

In [ ]:
# Connect to the HF Serverless Inference API
with open("../hf_token.txt", "r") as f:
    hf_token = f.readline()

login(token = hf_token)

We'll start creating an agent capable of searching the web using DuckDuckGo. We'll use the default model *Qwen/Qwen2.5-Coder-32B-Instruct*

In [ ]:
agent = CodeAgent(tools = [DuckDuckGoSearchTool()],
                  model = HfApiModel())
agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

The next step will be using the *@tool* decorator to define a custom funciton that acts as a tool.

In [ ]:
@tool
def suggest_menu(occasion:str) -> str:
    """Suggests a menu based on the occasion.
    Args:
        occasion: the type of occasion for the party."""
    
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

In [ ]:
agent = CodeAgent(tools = [suggest_menu],
                  model = HfApiModel())

agent.run("Prepare a formal menu for the party.")

*smolagents* specializes in agents that write and execute Python code snippets, offering sandboxed execution for security.

Code execution has strict security measures: imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in *additional_authorized_imports*.

In [ ]:
agent = CodeAgent(tools = [DuckDuckGoSearchTool(), suggest_menu],
                  model = HfApiModel(),
                  additional_authorized_imports = ["datetime"])

agent.run("""Alfred needs to prepare for the party. Here are the tasks:
          1. Prepare the drinks - 30 minutes
          2. Decorate the mansion - 60 minutes
          3. Set up the menu - 45 minutes
          4. Prepare the music and playlist - 45 minutes
          
          If we start right now, at what time will the party be ready?""")

## Pushing the code into Hugging Face Hub

In [ ]:
# Pushing the code to the Hub
hf_username = "germanebr"
agent.push_to_hub(f'{hf_username}/AlfredAgent')

In [ ]:
# Download an agent from a HF Hub repo
alfred_agent = agent.from_hub(f"{hf_username}/AlfredAgent")
alfred_agent.run("Give me the best playlist for a party at Wayne's mansion. the party idea is a 'villain masquerade' theme.")

# Full Alfred Agent

The following code lists a more complete agent prepared for performing multiple tasks apart from the ones mentioned above. Most of the unseen modules will be discussed later.

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool

In [ ]:
@tool
def suggest_menu(occasion:str) -> str:
    """Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party."""
    
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu from the butler."

In [ ]:
@tool
def catering_serviec_tool(query:str) -> str:
    """This tool returns the highest-rated catering service in Gotham City.
    Args:
        query: A search term for finding catering services."""
    
    # List of catering services and their ratings
    services = {"Gotham Catering Co.": 4.9,
                "Wayne Manor Catering": 4.8,
                "Gotham City Events": 4.7}
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services,
                       key = services.get)
    return best_service

In [ ]:
class SuperherPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    
    description = """This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {"category": {"type": "string",
                           "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham')."}}
    output_type = "string"

    def forward(self, category:str):
        themes = {"classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
                  "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
                  "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."}
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

In [ ]:
agent = CodeAgent(tools = [DuckDuckGoSearchTool(),
                           VisitWebpageTool(),
                           suggest_menu,
                           catering_serviec_tool,
                           SuperheroPartyThemeTool()],
                  model = HfApiModel(),
                  max_steps = 10,
                  verbosity_level = 2)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

## Inspecting the agent with OpenTelemetry and Langfuse

*smolagents* is capable to use the **OpenTelemetry** standard for instrumenting agent runs, allowing seamless inspection and logging. Apart from that, by using **Langfuse** and the **SmolagentsInstrumentor**, we can track and analyze the agent's behavior.

In [ ]:
# Install the dependencies
# pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [ ]:
import os
import base64
import json

from smolagents import CodeAgent, HfApiModel

from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [ ]:
with open("../langfuse_token.json", "r") as f:
    keys = json.load(f)

LANGFUSE_PUBLIC_KEY = keys["public_key"]
LANGFUSE_SECRET_KEY = keys["secret_key"]
LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [ ]:
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
agent = CodeAgent(tools=[], model=HfApiModel())
alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")